In [1]:
!gpustat

seven                Thu Mar  4 18:04:31 2021  440.64
[0] TITAN RTX        | 86'C,  83 % | 17140 / 24220 MB | sharuev(1649M) sharuev(12931M) sharuev(2549M)
[1] TITAN RTX        | 87'C,  98 % | 10230 / 24220 MB | cwb(3297M) cwb(3955M) cwb(2967M)
[2] TITAN RTX        | 55'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 86'C,  99 % |  9156 / 24220 MB | cwb(2917M) cwb(3053M) cwb(3175M)


In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

In [2]:
ddd = pd.read_csv('enru_train.csv')
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/new_ru_attacked.csv')).sample(frac=1, random_state=42)
#ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_attacked.csv'))
print(ddd.shape)
ddd.head()

(3397, 8)


,Unnamed: 0,Unnamed: 0.1,target,text,changed_words_num,new_model_target,old_model_target,old_text
291,291,131.0,A1,"Повседневное ТВ , ориентируясь на благополучие...",NaN,NaN,NaN,NaN
25,25,NaN,A8,"вторник , 12 июля 2011 года , 11.24 « ВКонтакт...",1.0,A7,A8,"вторник , 12 июля 2011 года , 11.24 « ВКонтакт..."
2334,887,839.0,A17,The Acoustic Music Co : Folk music instrument ...,NaN,NaN,NaN,NaN
432,432,361.0,A17,"Сабжекта не будет Жена купила журнал "" Идеи ва...",NaN,NaN,NaN,NaN
479,479,815.0,A16,Статистическая комиссия Тридцать четвертая сес...,NaN,NaN,NaN,NaN


In [3]:
ddd.to_csv('enru_attacked_train.csv')

Let's download all the data.

In [4]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_xlm_roberta_enru_attacked' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [5]:
transformer_model = 'xlm-roberta-base'
MAX_TOKENS = 512

In [6]:
tokenizer = PretrainedTransformerTokenizer(transformer_model)

03042021 18:10:05|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03042021 18:10:05|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03042021 18:10:06|INFO|transformers.tokenization_

In [7]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "enru_attacked_train.csv", 
    pathjoin(DATA_DIR, "manual_genres", "all.csv"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

03042021 18:10:09|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03042021 18:10:09|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03042021 18:10:09|INFO|transformers.tokenization_

03042021 18:10:19|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model from cache at /home/mlepekhin/.cache/torch/transformers/0c370616ddfc06067c0634160f749c2cf9d8da2c50e03a2617ce5841c8df3b1d.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed
03042021 18:10:20|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03042021 18:10:20|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
 

Reading data
<class 'data_processing.ClassificationDatasetReader'> enru_attacked_train.csv


03042021 18:15:45|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [8]:
model = build_pool_transformer_model(vocab, transformer_model)

Building the model


03042021 18:15:46|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03042021 18:15:46|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03042021 18:15:46|INFO|transformers.modeling_util

In [9]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [10]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [11]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

03042021 18:16:06|INFO|allennlp.training.optimizers| Number of trainable parameters: 278052107
03042021 18:16:06|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
03042021 18:16:06|INFO|allennlp.training.trainer| Beginning training.
03042021 18:16:06|INFO|allennlp.training.trainer| Epoch 0/9
03042021 18:16:06|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5792.84


Starting training


03042021 18:16:06|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 17140
03042021 18:16:06|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 10230
03042021 18:16:06|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 1990
03042021 18:16:06|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9156
03042021 18:16:06|INFO|allennlp.training.trainer| Training


03042021 18:18:03|INFO|allennlp.training.trainer| Validating


03042021 18:18:39|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.665  |     0.707
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  17140.000  |       N/A
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  10230.000  |       N/A
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  1990.000  |       N/A
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9156.000  |       N/A
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| loss               |     1.079  |     0.918
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03042021 18:18:39|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5792.840  |       N/A


...

03042021 18:40:10|INFO|allennlp.training.trainer| Epoch duration: 0:02:39.041847
03042021 18:40:10|INFO|allennlp.training.trainer| Estimated training time remaining: 0:02:40
03042021 18:40:10|INFO|allennlp.training.trainer| Epoch 9/9
03042021 18:40:10|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5792.84
03042021 18:40:11|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 17140
03042021 18:40:11|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 10230
03042021 18:40:11|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18470
03042021 18:40:11|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9156
03042021 18:40:11|INFO|allennlp.training.trainer| Training


03042021 18:42:09|INFO|allennlp.training.trainer| Validating


03042021 18:42:44|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.985  |     0.729
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  17140.000  |       N/A
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  10230.000  |       N/A
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  18470.000  |       N/A
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9156.000  |       N/A
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| loss               |     0.051  |     1.242
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03042021 18:42:44|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5792.840  |       N/A


03042021 18:42:49|INFO|allennlp.training.trainer| Epoch duration: 0:02:38.318316
03042021 18:42:49|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [12]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

03042021 18:42:51|WARNING|root| vocabulary serialization directory /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked/vocab is not empty
03042021 18:42:51|INFO|filelock| Lock 140228623927472 acquired on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked/vocab/.lock
03042021 18:42:51|INFO|filelock| Lock 140228623927472 released on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked/vocab/.lock


total 1,1G
-rw-rw-r-- 1 mlepekhin mlepekhin 1,1G мар  4 18:24 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K мар  4 18:16 log
